In [ ]:
import os 
import sys
import glob

sys.path.append("../data/utilities/spacenetutilities")

import geoTools as gT
import json
from osgeo import gdal, osr
from labeltools import pascalVOCLabel
import numpy as np
from labeltools import darkNetLabel

data_path = "/mnt/c/Users/tandon/OneDrive - TomTom/Desktop/tomtom/Workspace/01_Rooftop_accuracy/BFP-detection/data"
fullPathImageDirectory = os.path.join(data_path,"AOI_2_Vegas_Train/RGB-PanSharpen")
fullPathGeoJsonDirectory = os.path.join(data_path,"AOI_2_Vegas_Train/geojson/buildings")
outputdirectory = os.path.join(data_path,'output')

listofRaster = sorted(glob.glob(os.path.join(fullPathImageDirectory, '*.tif')))
listofgeojson = sorted(glob.glob(os.path.join(fullPathGeoJsonDirectory, '*.geojson')))

print("images :",len(listofRaster))
print("geojson:",len(listofgeojson))

if len(listofRaster) != len(listofgeojson):
    print('Error lists do not match fix source errors')

In [ ]:

def geojson_to_pixel_arr(raster_file, geojson_file, pixel_ints=True, verbose=False):
	'''
	Tranform geojson file into array of points in pixel (and latlon) coords
	pixel_ints = 1 sets pixel coords as integers
	'''
	
	# load geojson file
	with open(geojson_file) as f:
		geojson_data = json.load(f)

	# load raster file and get geo transforms
	src_raster = gdal.Open(raster_file)
	targetsr = osr.SpatialReference()
	targetsr.ImportFromWkt(src_raster.GetProjectionRef())
		
	geom_transform = src_raster.GetGeoTransform()
	
	# get latlon coords
	latlons = []
	types = []
	for feature in geojson_data['features']:
		coords_tmp = feature['geometry']['coordinates'][0]
		type_tmp = feature['geometry']['type']
		if verbose: 
			print("features:", feature.keys())
			print("geometry:features:", feature['geometry'].keys())

			#print("feature['geometry']['coordinates'][0]", z)
		latlons.append(coords_tmp)
		types.append(type_tmp)
		#print(feature['geometry']['type'])
	
	# convert latlons to pixel coords
	pixel_coords = []
	latlon_coords = []
	for i, (poly_type, poly0) in enumerate(zip(types, latlons)):
		
		if poly_type.upper() == 'MULTIPOLYGON':
			#print("oops, multipolygon")
			for poly in poly0:
				poly=np.array(poly)
				if verbose:
					print("poly.shape:", poly.shape)
					
				# account for nested arrays
				if len(poly.shape) == 3 and poly.shape[0] == 1:
					poly = poly[0]
					
				poly_list_pix = []
				poly_list_latlon = []
				if verbose: 
					print("poly", poly)
				for coord in poly:
					if verbose: 
						print("coord:", coord)
					lon, lat, z = coord 
					px, py = gT.latlon2pixel(lat, lon, input_raster=src_raster,
										 targetsr=targetsr, 
										 geom_transform=geom_transform)
					poly_list_pix.append([px, py])
					if verbose:
						print("px, py", px, py)
					poly_list_latlon.append([lat, lon])
				
				if pixel_ints:
					ptmp = np.rint(poly_list_pix).astype(int)
				else:
					ptmp = poly_list_pix
				pixel_coords.append(ptmp)
				latlon_coords.append(poly_list_latlon)            

		elif poly_type.upper() == 'POLYGON':
			poly=np.array(poly0)
			if verbose:
				print("poly.shape:", poly.shape)
				
			# account for nested arrays
			if len(poly.shape) == 3 and poly.shape[0] == 1:
				poly = poly[0]
				
			poly_list_pix = []
			poly_list_latlon = []
			if verbose: 
				print("poly", poly)
			for coord in poly:
				if verbose: 
					print("coord:", coord)
				lon, lat, z = coord 
				px, py = gT.latlon2pixel(lat, lon, input_raster=src_raster,
									 targetsr=targetsr, 
									 geom_transform=geom_transform)
				poly_list_pix.append([px, py])
				if verbose:
					print("px, py", px, py)
				poly_list_latlon.append([lat, lon])
			
			if pixel_ints:
				ptmp = np.rint(poly_list_pix).astype(int)
			else:
				ptmp = poly_list_pix
			pixel_coords.append(ptmp)
			latlon_coords.append(poly_list_latlon)
			
		else:
			print("Unknown shape type in coords_arr_from_geojson()")
			return
			
	return pixel_coords, latlon_coords

def processRasterChip(rasterImage, rasterDescription, geojson, geojsonDescription, outputDirectory='',
                      imagePixSize=-1, clipOverlap=0.0, randomClip=False,
                      minpartialPerc=0.0,
                      outputPrefix=''):

    # cut Image to Size
    chipSummaryList=[]
    if imagePixSize>0:

        rasterFileList = [[rasterImage, rasterDescription]]
        shapeFileSrcList = [[geojson, geojsonDescription]]
        # cut image to size
        print(rasterFileList)
        chipSummaryList = gT.cutChipFromMosaic(rasterFileList,
                                           shapeFileSrcList,
                                           outputDirectory=outputDirectory,
                                           outputPrefix=outputPrefix,
                                           clipSizeMX=imagePixSize,
                                           clipSizeMY=imagePixSize,
                                           minpartialPerc=minpartialPerc,
                                           createPix=True,
                                           clipOverlap=clipOverlap,
                                           noBlackSpace=True,
                                           randomClip=-1
                                           )
    else:
        chipSummary =  {'rasterSource': rasterImage,
                   'chipName': rasterImage,
                   'geoVectorName': geojson,
                   'pixVectorName': ''
                   }

        chipSummaryList.append(chipSummary)
    return chipSummaryList


def processChipSummaryList(chipSummaryList, outputDirectory='', annotationType='PASCALVOC2012', outputFormat='GTiff',
                           outputPixType='',
                           datasetName='spacenetV2',
                           folder_name='folder_name',
                           bboxResize=1.0
                           ):

    if outputPixType == '':
        convertTo8Bit = False
    else:
        convertTo8Bit = True

    entryList = []
    for chipSummary in chipSummaryList:



        annotationName = os.path.basename(chipSummary['rasterSource'])
        annotationName = annotationName.replace('.tif', '.xml')
        annotationName = os.path.join(outputDirectory, annotationName)



        if annotationType=='PASCALVOC2012':
            entry = pascalVOCLabel.geoJsonToPASCALVOC2012(annotationName, chipSummary['geoVectorName'], chipSummary['rasterSource'],
                                                          dataset='spacenetV2',
                                                          folder_name='spacenetV2',
                                                          annotationStyle=annotationType,
                                                          segment=True,
                                                          bufferSizePix=2.5,
                                                          convertTo8Bit=convertTo8Bit,
                                                          outputPixType=outputPixType,
                                                          outputFormat=outputFormat,
                                                          bboxResize=bboxResize
                                                          )
        elif annotationType=='DARKNET':
            entry = darkNetLabel.geoJsonToDARKNET(annotationName, chipSummary['geoVectorName'], chipSummary['rasterSource'],
                                                  dataset='spacenetV2',
                                                  folder_name='spacenetV2',
                                                  annotationStyle=annotationType,
                                                  convertTo8Bit=convertTo8Bit,
                                                  outputPixType=outputPixType,
                                                  outputFormat=outputFormat,
                                                  bboxResize=bboxResize
                                                  )
        else:

            print("Annotation Type = {} is not supported yet".format(annotationType))
            break
        entryList.append(entry)
    return entryList

In [ ]:
entryList = []
for rasterImage, geoJson in zip(listofRaster, listofgeojson):
    chipSummaryList = processRasterChip(rasterImage, "RGB-PanSharpen",geoJson, "geojson/buildings",outputDirectory=outputdirectory,imagePixSize=400, clipOverlap=0.0, randomClip=False,minpartialPerc=0.0,outputPrefix='')
    entryListTmp = processChipSummaryList(chipSummaryList,
                    outputDirectory=os.path.join(outputdirectory, 'annotations'),annotationType="DARKNET",outputFormat="JPEG",outputPixType="Byte",
                                        datasetName='spacenetV2',folder_name='folder_name',bboxResize= 1.0)
    print(entryListTmp)
    entryList.extend(entryListTmp)
                                                    